In [1]:
# Lab 11 MNIST and Convolutional Neural Network
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

'1.0.1.post2'

In [2]:
# cuda가 사용가능한 경우, cuda를 활용하겠다는 뜻
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
# torchvision에 있는 dataset인 dsets 에서 가져올 것이다.
# 앞의 강의의 주소에 이미 다운로드 되어 있으므로, 그것을 불러오고자 함.
# transform : input data를 어떻게 변형할지 --> tensor value로 전환해서 쓰겠다는 뜻
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
# shuffle : dataset을 섞은 상태로 던져줘라

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    # __init__(self) 를 빼먹으면 학습이 되지 않는다.
    def __init__(self):
        super(CNN, self).__init__()
        # layer1에서 사용될 층들 nn.Sequential로 정의
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        
        # layer2에서 사용될 층들 nn.Sequential로 정의
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # fully connected layer
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        # fully connected layer만 우선 초기화해줌
        torch.nn.init.xavier_uniform_(self.fc.weight)

        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [7]:
# instantiate CNN model
# model을 instance화 시킴
model = CNN().to(device)

In [8]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# train my model 학습코드!!
# 컴퓨터 사양 때문에 주피터노트북이 끊기는 것 같은느낌 ㅜ_ㅜ
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        # cuda로 하는 경우 cuda Tensor로 되어야 하므로 필수 사항임
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        # input 값을 model에 넣었을 때의 출력값
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        # 학습 진행
        optimizer.step()

        # cost를 total_batch로 나누어서 쌓아줌
        # epoch에서 cost가 몇이었는지 확인 가능
        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.


In [ ]:
# Test model and check accuracy
# test 시에는 grad 축적하지 않음 
with torch.no_grad():
    # test 데이터 불러옴
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())